## Assigmnent 3

In [1]:
from PattRecClasses import DiscreteD, GaussD, HMM, MarkovChain
from matplotlib import pyplot as plt

import os
import numpy as np

In [1]:
#https://www.kaggle.com/datasets/avk256/activity-recognition

### Feature extraction

In [3]:

acc = []
tau = [] # sample index for first sample in each recording

for filename in os.listdir('sensorlogger'):
    path = os.path.join('sensorlogger',filename)
    with open(path) as file:        
        if os.path.isfile(path):
            lines = [line.split() for line in file]
            # abs of norm of acceleration - gravity
            acc.extend([ np.abs((np.sqrt(float(line[2])**2 + float(line[3])**2 + float(line[4])**2))-9.82) for line in lines ])
            tau.append(len(acc)+1)
        
        
#t = [(float(line[0])-float(lines[0][0]))/1000 for line in lines]

%matplotlib qt
plt.figure(figsize= (15,10))
plt.subplot(311)
plt.plot(acc)



qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.



** (python3.10:6849): WARNING **: 13:53:47.312: atk-bridge: get_device_events_reply: unknown signature


In [12]:
"""Windowing"""
wLen = 100 # window length (samples)
overlap = 0 # is higher resolution needed?
windows = []

step = wLen-overlap

X = []
for n in range(0,len(acc)-wLen +1,step):
    X.append(sum(acc[n:n+wLen])/wLen)
    windows.extend([sum(acc[n:n+wLen])/wLen]*wLen)
    
"""Labels"""
S = []

"""Cluster training data into N separate categories, for estimation of output probabilities"""
data1 = []
data2 = []
data3 = []

for a in X:
    if a > 3.5:
        S.append(3) #running
        data1.append(a)
    elif a > 0.6:
        S.append(2)  #walking
        data2.append(a)
    else:
        S.append(1)  #standing
        data3.append(a)

colors = {1: 'red', 2: 'green', 3: 'blue'}

plt.subplot(312)
plt.plot(windows)

plt.subplot(313)
plt.plot([t2 for t2 in range(0, len(acc), wLen)][:-1], X)
for i, val in enumerate(S):
    plt.scatter(i*wLen, val, color = colors[val])

plt.show()


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


In [5]:
"""Extracting data to csv"""

datapoints = np.array([X,S]).T

import csv

with open('features.csv', 'w') as csvfile:
    Writer = csv.writer(csvfile)
    Writer.writerow(tau)
    for data in datapoints:
        Writer.writerow(data)



In [16]:
"""Initialization"""

"""Infinite ergodic HMM may start in any state. Assumption here is that
    each state is equally likely to occur as initial state"""

mc = MarkovChain( np.full( (1,3),1/3 ) , np.array( [ [ 0.9, 0.1, 0 ], 
                                                  [ 0.05, 0.9, 0.05 ], 
                                                  [ 0, 0.05 ,0.9] ] ) )

"""The diagonal elements of the transition matrix can be chosen to indicate 
    approximately the average state durations. Here I assume that each state duration is equally long"""


"""Output probability distributions must be initialized differently for each state"""
# Maybe do this with separate data?

from scipy.stats import norm
        
mu1, std1 = norm.fit(data1)
mu2, std2 = norm.fit(data2)
mu3, std3 = norm.fit(data3)


""" Looks reasonable
print(mu1,std1)
print(mu2,std2)
print(mu3,std3)
"""

b1 = GaussD( means=[mu1], stdevs=[std1] )   # Distribution for state = standing
b2 = GaussD( means=[mu2], stdevs=[std2] )   # Distribution for state = walking
b3 = GaussD( means=[mu3], stdevs=[std3] )   # Distribution for state = running

"""Initial HMM"""

#h0  = HMM( mc, [g1, g2])

5.945451797660629 0.9650501323041633
1.7236981774924978 0.4553284434081767
0.20466219393869484 0.1085833637535468


'Initial HMM'

### Training

In [ ]:
"""Updating initial probability vector"""

x_obs = (-0.2, 2.6, 1.3)
pX = np.array([b1.prob(x_obs), b2.prob(x_obs)])
sf = [np.max(p) for p in pX.T]
pX = pX/sf
    
[alfaHat, c] = mc.forward(pX)

betaHat = mc.backward(pX, c)
